In [1]:
import numpy as np
import random
from sklearn.neural_network import MLPClassifier

In [2]:
def split_train_val_id(n, val_percent=0.1):
    ids = [i for i in range(n)]
    random.shuffle(ids)
    piv = int(n*val_percent)
    if piv==0: 
        piv=1
    return {'train': ids[:-piv], 'val': ids[-piv:]}

In [3]:
datas = np.load('train_features.npy')
labels = np.load('train_labels.npy')

In [16]:
NUM_DATA = 5000
INPUT_DIM_1D = datas.shape[1]*datas.shape[2]
OUTPUT_DIM = labels.shape[1]-1
val_percent = 0.2

train_val_id = split_train_val_id(NUM_DATA, val_percent)
train_id = train_val_id['train']
val_id = train_val_id['val']

train_datas = np.zeros((len(train_id), INPUT_DIM_1D))
train_labels = np.zeros((len(train_id), OUTPUT_DIM))
for i, idx in enumerate(train_id):
    train_datas[i] = datas[idx].reshape(INPUT_DIM_1D)
    train_labels[i] = labels[idx][1:]
    
#reduce data information
train_datas = train_datas[:, :10000]
#more than n time slots == 1, predict 1
labels_onehot = np.zeros(len(train_id))
for i in range(len(train_id)):
    if train_labels[i].sum()>=3:
        labels_onehot[i] = 1

In [17]:
_solver = 'adam'
_activation = 'relu'
_alpha = 1e-4
_hidden_layer = (64, 32)
_lr_rate = 0.001
_batch = 64
_max_iter = 200

mlp = MLPClassifier(solver=_solver, activation=_activation, alpha=_alpha, hidden_layer_sizes=_hidden_layer,\
                    batch_size=_batch, learning_rate_init=_lr_rate, max_iter=_max_iter)
mlp.fit(train_datas, labels_onehot)


MLPClassifier(activation='relu', alpha=0.0001, batch_size=64, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(64, 32), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [7]:
from sklearn.metrics import classification_report

In [14]:
NUM_OF_VAL = len(val_id)
val_datas = np.zeros((NUM_OF_VAL, INPUT_DIM_1D))
val_labels = np.zeros((NUM_OF_VAL, OUTPUT_DIM))
for i, idx in enumerate(val_id):
    val_datas[i] = datas[idx].reshape(INPUT_DIM_1D)
    val_labels[i] = labels[idx][1:]
    
#reduce data information
val_datas = val_datas[:, :10000]
# more than n time slots == 1, predict 1
val_labels_onehot = np.zeros(NUM_OF_VAL)
for i in range(NUM_OF_VAL):
    if val_labels[i].sum()>=3:
        val_labels_onehot[i] = 1

In [15]:
y_pred = mlp.predict(val_datas)
print(classification_report(val_labels_onehot, y_pred)) 

              precision    recall  f1-score   support

         0.0       0.93      0.92      0.93       735
         1.0       0.78      0.82      0.80       265

    accuracy                           0.89      1000
   macro avg       0.86      0.87      0.86      1000
weighted avg       0.89      0.89      0.89      1000



In [23]:
cnt = 0
n = 3
for i in range(NUM_DATA):
    if labels[i].sum()>=n:
        cnt+=1
print('{} of the {} have more than {} time slots ==1.'.format(cnt, labels.shape[0], n))

3998 of the 10065 have more than 3 time slots ==1.
